<a href="https://colab.research.google.com/github/BillelBenoudjit/CLA_Bootcamp/blob/main/CamemBERT_fine_tuning_with_Keras_(GPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install "tensorflow==2.6.0"
!pip install transformers[sentencepiece] datasets seqeval tensorboard --upgrade

     |████████████████████████████████| 3.4 MB 24.7 MB/s 
     |████████████████████████████████| 311 kB 53.8 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 5.8 MB 56.5 MB/s 
     |████████████████████████████████| 133 kB 74.5 MB/s 
     |████████████████████████████████| 1.1 MB 63.8 MB/s 
     |████████████████████████████████| 243 kB 76.0 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 64.4 MB/s 
     |████████████████████████████████| 271 kB 79.4 MB/s 
     |████████████████████████████████| 895 kB 70.7 MB/s 
     |████████████████████████████████| 596 kB 73.7 MB/s 
     |████████████████████████████████| 3.3 MB 52.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=e3e9bfb2485aba0e7a7612c930c8fe5d803d32fa0f0a26cc5

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
model_id = "jplu/tf-camembert-base"

In [4]:
dataset_id = "wikiann"
seed = 33

In [5]:
from datasets import load_dataset
dataset = load_dataset(dataset_id, 'fr')

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
ner_labels = dataset['train'].features['ner_tags'].feature.names
print(ner_labels)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [8]:
print(dataset['train'])

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 20000
})


In [9]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples["tokens"], truncation=True, is_split_into_words=True
  )
  labels = []
  for i, label in enumerate(examples[f"ner_tags"]):
    # Get the list of tokens their connecting word id (for words tokenized into multiple chunks)
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      # Special tokens have a word id that is None, we set the label to -100
      # so they are automatically ignored in the loss function
      if word_idx is None:
        label_ids.append(-100)
      # We set the label for the first token of each word
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      # For the other tokens in a word, we set the label to the current one
      else:
        label_ids.append(label[word_idx])
      previous_word_idx = word_idx        
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [12]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-5540ab024efc3734.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-a5ef12fb6b88aecf.arrow


In [13]:
print(tokenized_datasets['test'])
print(tokenized_datasets['train'])

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})
Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [14]:
pre_tokenizer_columns = set(dataset['train'].features)
print(pre_tokenizer_columns)
tokenizer_columns = list(set(tokenized_datasets['train'].features) - pre_tokenizer_columns)
print(tokenizer_columns)

{'ner_tags', 'langs', 'spans', 'tokens'}
['labels', 'attention_mask', 'input_ids']


In [15]:
processed_dataset = tokenized_datasets.shuffle(seed=seed)
processed_dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [16]:
# Set Hyperparameters
from huggingface_hub import HfFolder
import tensorflow as tf

id2label = {str(i): label for i, label in enumerate(ner_labels)}
print(id2label)
label2id = {v: k for k, v in id2label.items()}
print(label2id)

num_train_epochs = 5
train_batch_size = 16
eval_batch_size = 32
learning_rate = 2e-5
weight_decay_rate = 0.01
num_warmup_steps = 0
output_dir = model_id.split("/")[0]
print(output_dir)
hub_token = HfFolder.get_token()
hub_model_id = f'{model_id.split("/")[0]}-{dataset_id}'
print(hub_model_id)

fp16=True 
# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
if fp16:
  tf.keras.mixed_precision.set_global_policy("mixed_float16")

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6'}
jplu
jplu-wikiann
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [17]:
# Convert the dataset to tf.data.Dataset
from transformers import DataCollatorForTokenClassification

# Data collator that will dynamically pad the inputs received, as well as the labels
data_collator = DataCollatorForTokenClassification(
    tokenizer = tokenizer,
    return_tensors = "tf"
)

# Convert the train dataset to tf.data.Dataset
tf_train_dataset = processed_dataset["train"].to_tf_dataset(
    columns = tokenizer_columns,
    shuffle = False,
    batch_size = train_batch_size,
    collate_fn = data_collator
)

# Convert the test dataset to tf.data.Dataset
tf_test_dataset = processed_dataset["test"].to_tf_dataset(
    columns = tokenizer_columns,
    shuffle = False,
    batch_size = eval_batch_size,
    collate_fn = data_collator
)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [18]:
def connect_to_tpu(tpu_address: str = None):
  if tpu_address is not None:  # When using GCP
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu=tpu_address
      )
    if tpu_address not in ("", "local"):
      tf.config.experimental_connect_to_cluster(cluster_resolver)
      tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
      strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
      print("Running on TPU ", cluster_resolver.master())
      print("REPLICAS: ", strategy.num_replicas_in_sync)
      return cluster_resolver, strategy
  else: # When using Colab or Kaggle
    try:
      cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
      strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
      print("Running on TPU ", cluster_resolver.master())
      print("REPLICAS: ", strategy.num_replicas_in_sync)
      return cluster_resolver, strategy
    except:
      print("WARNING: No TPU detected.")
      mirrored_strategy = tf.distribute.MirroredStrategy()
      return None, mirrored_strategy

In [19]:
# Download the pretrained transformer and fine-tune it
from transformers import TFAutoModelForTokenClassification, create_optimizer

num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr = learning_rate,
    num_train_steps = num_train_steps,
    weight_decay_rate = weight_decay_rate,
    num_warmup_steps = num_warmup_steps
)

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# cluster_resolver, strategy = connect_to_tpu()

# with strategy.scope():
model = TFAutoModelForTokenClassification.from_pretrained(
    model_id,
    id2label = id2label,
    label2id = label2id
  )
model.compile(optimizer=optimizer)

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFCamembertForTokenClassification.

Some layers of TFCamembertForTokenClassification were not initialized from the model checkpoint at jplu/tf-camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [22]:
# Push data to the hub
import os
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

callbacks = []
callbacks.append(TensorBoard(log_dir=os.path.join(output_dir,"logs")))

if hub_token:
  callbacks.append(PushToHubCallback(
      output_dir = output_dir,
      tokenizer = tokenizer,
      hub_model_id = hub_model_id,
      hub_token = hub_token
  ))

Cloning https://huggingface.co/BillelBenoudjit/jplu-wikiann into local empty directory.


In [21]:
#!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 2s (1,212 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories c

In [23]:
#print(tf_train_dataset)

model.fit(
    tf_train_dataset,
    validation_data = tf_test_dataset,
    callbacks = callbacks,
    epochs = 5
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:377: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


1250/1250 [==============================] - 194s 137ms/step - loss: 0.6059 - val_loss: 0.3086 - epoch: 0.0000e+00
Epoch 2/5
   1/1250 [..............................] - ETA: 3:11 - loss: 0.2532

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


1250/1250 [==============================] - 175s 140ms/step - loss: 0.2659 - val_loss: 0.2371 - epoch: 1.0000
Epoch 3/5
1250/1250 [==============================] - 169s 135ms/step - loss: 0.1878 - val_loss: 0.2098 - epoch: 2.0000
Epoch 4/5
1250/1250 [==============================] - 155s 124ms/step - loss: 0.1463 - val_loss: 0.2010 - epoch: 3.0000
Epoch 5/5
1250/1250 [==============================] - 155s 124ms/step - loss: 0.1251 - val_loss: 0.2025 - epoch: 4.0000


KeyboardInterrupt: ignored

In [25]:
from datasets import load_metric
import numpy as np

metric = load_metric("seqeval")

def evaluate(model, dataset, ner_labels):
  all_predictions = []
  all_labels = []
  for batch in dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis = -1)
    for prediction, label in zip(predictions, labels):
      for predicted_idx, label_idx in zip(prediction, label):
        if label_idx == -100:
          continue
        #print(f"Predicted_idx ner_labels[{predicted_idx}] = {ner_labels[predicted_idx]} / Label_idx ner_labels[{label_idx}] = {ner_labels[label_idx]}")
        all_predictions.append(ner_labels[predicted_idx])
        all_labels.append(ner_labels[label_idx])
  return metric.compute(predictions=[all_predictions], references=[all_labels])

results = evaluate(model, tf_test_dataset, ner_labels=list(model.config.id2label.values()))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:377: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)


In [26]:
import pprint

pprint.pprint(results)

{'LOC': {'f1': 0.9180855010924682,
         'number': 12034,
         'precision': 0.9109874826147427,
         'recall': 0.9252949975070633},
 'ORG': {'f1': 0.8364760899079801,
         'number': 6565,
         'precision': 0.8284775138204094,
         'recall': 0.8446306169078446},
 'PER': {'f1': 0.938287153652393,
         'number': 7925,
         'precision': 0.9365179132620993,
         'recall': 0.9400630914826499},
 'overall_accuracy': 0.9464171271196716,
 'overall_f1': 0.9038299466242158,
 'overall_precision': 0.897994120055078,
 'overall_recall': 0.9097421203438395}


In [27]:
from transformers.modelcard import TrainingSummary


eval_results = {
    "precision":float(results["overall_precision"]),
    "recall":float(results["overall_recall"]),
    "f1":float(results["overall_f1"]),
    "accuracy":float(results["overall_accuracy"]),
}

training_summary = TrainingSummary(
    model_name = hub_model_id,
    language = "fr",
    tags=[],
    finetuned_from=model_id,
    tasks="token-classification",
    dataset=dataset_id,
    dataset_tags=dataset_id,
    dataset_args="default",
    eval_results=eval_results,
    hyperparameters={
        "num_train_epochs": num_train_epochs,
        "train_batch_size": train_batch_size,
        "eval_batch_size": eval_batch_size,
        "learning_rate": learning_rate,
        "weight_decay_rate": weight_decay_rate,
        "num_warmup_steps": num_warmup_steps,
        "fp16": fp16
    }
)

model_card = training_summary.to_model_card()

model_card_path = os.path.join(output_dir, "README.md")

with open(model_card_path, "w") as f:
    f.write(model_card)

In [28]:
from huggingface_hub import HfApi

api = HfApi()

user = api.whoami(hub_token)

api.upload_file(
    token=hub_token,
    repo_id=f"{user['name']}/{hub_model_id}",
    path_or_fileobj=model_card_path,
    path_in_repo="README.md",
)

'https://huggingface.co/BillelBenoudjit/jplu-wikiann/blob/main/README.md'